In [49]:
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
!pip install wurlitzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

In [52]:
#@title

from IPython.core.magic import register_line_magic
from IPython.display import Javascript
from IPython.display import display as ipy_display

# Some of the model training logs can cover the full
# screen if not compressed to a smaller viewport.
# This magic allows setting a max height for a cell.
@register_line_magic
def set_cell_height(size):
  ipy_display(
      Javascript("google.colab.output.setIframeHeight(0, true, {maxHeight: " +
                 str(size) + "})"))

Загрузка датасета

In [53]:
# Load a dataset into a Pandas Dataframe.
dataset_df = pd.read_csv("/content/boston.csv")

# Display the first 3 examples.
dataset_df.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7


Функция для разделения датасета на выборку для обучения и выборку для тестирования (в соотношении 70 к 30)

In [54]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

344 examples in training, 162 examples for testing.


Поместить в лэйбл имя предсказываемого параметра

In [55]:
# Name of the label column.
label = "MEDV"

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

Обучение модели на регрессию

In [56]:
%set_cell_height 300

# Configure the model.
model_RF = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)

# Train the model.
model_RF.fit(x=train_ds)

<IPython.core.display.Javascript object>

Use /tmp/tmph_p9l1ch as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.270994. Found 344 examples.
Training model...
Model trained in 0:00:00.353946
Compiling model...
Model compiled.


Сохранение модели

In [57]:
model_RF.save("/tmp/my_saved_model")

Оценка модели по MSE и RMSE

In [58]:
# Evaluate the model on the test dataset.
model_RF.compile(metrics=["mse"])
evaluation = model_RF.evaluate(test_ds, return_dict=True)

print(evaluation)
print()
print(f"MSE: {evaluation['mse']}")
print(f"RMSE: {math.sqrt(evaluation['mse'])}")
tfdf.model_plotter.plot_model_in_colab(model_RF, tree_idx=100, max_depth=4)

#model_RF.make_inspector().variable_importances()

1/1 [==============================] - 0s 158ms/step - loss: 0.0000e+00 - mse: 12.5101
{'loss': 0.0, 'mse': 12.510115623474121}

MSE: 12.510115623474121
RMSE: 3.536964181819505


Вывод различной информации о модели

In [59]:
%set_cell_height 300
model_RF.summary()

<IPython.core.display.Javascript object>

Model: "random_forest_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (13):
	AGE
	B
	CHAS
	CRIM
	DIS
	INDUS
	LSTAT
	NOX
	PTRATIO
	RAD
	RM
	TAX
	ZN

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1. "__LABEL"  7.054064 ################
    2.    "CHAS"  7.016923 ###############
    3.      "ZN"  6.870783 ###############
    4.     "RAD"  6.655474 ##############
    5.       "B"  6.249579 #############
    6.     "TAX"  6.180400 #############
    7.   "INDUS"  5.692491 ###########
    8.     "AGE"  5.615337 ###########
    9. "PTRATIO"  5.391385 ###########
   10.     "DIS"  5.285461 ##########
   11.    "CRIM"  4.874893 #########
   12.     "NOX"  4.824924 #########
   13.      "RM"  2.508343 ##

In [60]:
model_RF.make_inspector().features()

["AGE" (1; #0),
 "B" (1; #1),
 "CHAS" (1; #2),
 "CRIM" (1; #3),
 "DIS" (1; #4),
 "INDUS" (1; #5),
 "LSTAT" (1; #6),
 "NOX" (1; #7),
 "PTRATIO" (1; #8),
 "RAD" (1; #9),
 "RM" (1; #10),
 "TAX" (1; #11),
 "ZN" (1; #12)]

In [61]:

%set_cell_height 150
model_RF.make_inspector().training_logs()

<IPython.core.display.Javascript object>

[TrainLog(num_trees=1, evaluation=Evaluation(num_examples=140, accuracy=None, loss=None, rmse=4.096586100863651, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=11, evaluation=Evaluation(num_examples=343, accuracy=None, loss=None, rmse=4.283912403897084, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=21, evaluation=Evaluation(num_examples=344, accuracy=None, loss=None, rmse=3.888518172013881, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=31, evaluation=Evaluation(num_examples=344, accuracy=None, loss=None, rmse=3.7398301884691767, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=41, evaluation=Evaluation(num_examples=344, accuracy=None, loss=None, rmse=3.7768026038974427, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=51, evaluation=Evaluation(num_examples=344, accuracy=None, loss=None, rmse=3.73410361446723, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=61, evaluation=Evalu